# DATA COLLECTION


In [2]:
import pandas as pd 
import numpy as np
import requests
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import rasterio 
import matplotlib.pyplot as plt
import dask.array as da
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:

tif_file_path = 'C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2013.tif'

with rasterio.open(tif_file_path) as src:
    
    raster_data = src.read()
    metadata = src.meta
print("Raster Shape:", raster_data.shape)
print("Metadata:", metadata)


Raster Shape: (1, 12839, 14816)
Metadata: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 14816, 'height': 12839, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.000509090909090909, 0.0, 77.09621818181817,
       0.0, -0.0005090909090909091, 30.408)}


In [4]:
def read_lulc_images(file_paths):
    lulc_data = []
    
    for file_path in file_paths:
        with rasterio.open(file_path) as src:

            data = src.read(1)  
            
        
            lulc_data.append(data)
    
    return lulc_data

lulc_file_paths = ['lulc_image1.tif', 'lulc_image2.tif', 'lulc_image3.tif']
lulc_images = read_lulc_images(lulc_file_paths)


RasterioIOError: lulc_image1.tif: No such file or directory

In [ ]:

pixel_value = raster_data[:, 1000,1000]
print("Pixel Value:", pixel_value)


In [ ]:
subset_raster_data = raster_data[:, :1000, :1000] 
statistics = {'min': subset_raster_data.min(), 'max': subset_raster_data.max(), 'mean': subset_raster_data.mean(), 'std': subset_raster_data.std()}
print("Statistics:", statistics)


In [ ]:
downsampled_raster_data = raster_data[:, ::2, ::2]  # Adjust the downsampling factor as needed
statistics = {'min': downsampled_raster_data.min(), 'max': downsampled_raster_data.max(), 'mean': downsampled_raster_data.mean(), 'std': downsampled_raster_data.std()}
print("Statistics:", statistics)


In [ ]:

dask_raster_data = da.from_array(raster_data, chunks=(1, 1000, 1000))
statistics = {'min': dask_raster_data.min().compute(), 'max': dask_raster_data.max().compute(), 'mean': dask_raster_data.mean().compute(), 'std': dask_raster_data.std().compute()}
print("Statistics:", statistics)


In [ ]:

plt.figure(figsize=(8, 8))
plt.imshow(raster_data[0, :, :], cmap='viridis') 
plt.colorbar(label='Pixel Value')
plt.title('Remote Sensing Image')
plt.show()
print("Metadata:", metadata)



# DATA PREPROCESSING

In [ ]:
from rasterio.enums import Resampling

def resample_raster(input_path, output_path, scale_factor):
    with rasterio.open(input_path) as src:
        data = src.read(
            out_shape=(src.count, int(src.height * scale_factor), int(src.width * scale_factor)),
            resampling=Resampling.bilinear)

        profile = src.profile
        profile.update(width=int(src.width * scale_factor), height=int(src.height * scale_factor))

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data)


In [ ]:
from rasterio.windows import Window

def crop_raster(input_path, output_path, window):
    with rasterio.open(input_path) as src:
        data = src.read(window=window)

        profile = src.profile
        profile.update(width=window.width, height=window.height, transform=src.window_transform(window))

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data)


In [ ]:
def normalize_raster(input_path, output_path, min_value, max_value):
    with rasterio.open(input_path) as src:
        data = src.read()
        data = ((data - data.min()) / (data.max() - data.min())) * (max_value - min_value) + min_value

        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data)


In [ ]:
def calculate_ndvi(input_path, output_path):
    with rasterio.open(input_path) as src:
        red_band = src.read(3)
        nir_band = src.read(4)

        ndvi = (nir_band - red_band) / (nir_band + red_band)

        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(ndvi, 1)


In [ ]:
def stack_bands(input_paths, output_path):
    with rasterio.open(input_paths[0]) as src:
        profile = src.profile

    stack_data = [rasterio.open(path).read() for path in input_paths]

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(stack_data)


In [ ]:
def calculate_lulc_percentages(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read()

        # Assuming each band corresponds to a different LULC class
        num_classes = src.count

        # Calculate percentage for each class
        class_percentages = (data / data.sum(axis=0)) * 100

        profile = src.profile
        profile.update(count=num_classes)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(class_percentages)


In [ ]:
def calculate_dominant_lulc(input_path, output_path):
    with rasterio.open(input_path) as src:
        data = src.read()

        # Assuming each band corresponds to a different LULC class
        dominant_class = np.argmax(data, axis=0) + 1  # Add 1 to make class indices start from 1

        profile = src.profile

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(dominant_class)


In [ ]:


gdp_data = pd.read_csv('C:\\Users\\my\\Documents\\GDP.csv')


print(gdp_data.head())  



In [ ]:
print(gdp_data.tail())

In [ ]:
gdp_data['GDP'].fillna(gdp_data['GDP'].mean(), inplace=True)


In [ ]:
import rasterio
from rasterio.plot import show

# Replace 'your_satellite_image.tif' with the path to your satellite image GeoTIFF file
satellite_image_path = 'C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2012.tif'

# Open the satellite image GeoTIFF file
with rasterio.open(satellite_image_path) as src:
    # Read all bands of the satellite image
    satellite_data = src.read()

    # Access metadata
    metadata = src.meta

# Display the RGB satellite image using rasterio.plot
show(satellite_data, cmap='viridis', title='RGB Satellite Image')


In [ ]:
import rasterio
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt

# Replace 'your_raster_file.tif' with the path to your GeoTIFF file
raster_file_path ="C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2012.tif"

# Open the GeoTIFF file
with rasterio.open(raster_file_path) as src:
    # Read the raster data
    raster_data = src.read(1)  # Assuming you are reading the first band

    # Flatten the 2D array to 1D for z-score calculation
    flat_data = raster_data.flatten()

    # Calculate z-scores for each pixel value
    z_scores = zscore(flat_data)

    # Define a threshold for considering values as outliers (e.g., z-score > 3 or < -3)
    threshold = 3

    # Identify outlier indices
    outlier_indices = np.where(np.abs(z_scores) > threshold)[0]

    # Display the histogram of z-scores
    plt.hist(z_scores, bins=50, density=True, alpha=0.75, color='b')
    plt.axvline(threshold, color='r', linestyle='dashed', linewidth=2, label=f'Outlier Threshold ({threshold})')
    plt.axvline(-threshold, color='r', linestyle='dashed', linewidth=2)
    plt.xlabel('Z-Score')
    plt.ylabel('Frequency')
    plt.legend()
    plt.title('Histogram of Z-Scores')
    plt.show()

# Print indices of potential outliers
print("Indices of potential outliers:", outlier_indices)


# LULC AND GDP

Markov chain model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
transition_matrix = np.array([
    [0.7, 0.2, 0.1],
    [0.3, 0.6, 0.1],
    [0.1, 0.1, 0.8]
])

In [ ]:
# Initial distribution of land cover classes
initial_distribution = np.array([0.4, 0.4, 0.2])

In [ ]:
# Simulate transitions over time steps
num_steps = 1
land_cover_states = np.zeros((num_steps + 1, len(initial_distribution)))
land_cover_states[0] = initial_distribution


In [ ]:
for t in range(1, num_steps + 1):
    land_cover_states[t] = np.dot(land_cover_states[t - 1], transition_matrix)


In [ ]:
# Simulated GDP values associated with each land cover state
gdp_values = np.array([120, 100, 200])

In [ ]:
# Calculate GDP over time based on simulated land cover states
gdp_over_time = np.dot(land_cover_states, gdp_values)


In [ ]:
# Create a DataFrame for visualization
df = pd.DataFrame(land_cover_states, columns=[])
df['GDP'] = gdp_over_time


In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df['GDP'], label='GDP')
plt.plot(df['LULC_State_1'], label='LULC State 1')
plt.plot(df['LULC_State_2'], label='LULC State 2')
plt.plot(df['LULC_State_3'], label='LULC State 3')
plt.xlabel('Time Steps')
plt.ylabel('Values')
plt.legend()
plt.title('Simulation of LULC and GDP Over Time')
plt.show()

In [ ]:
# Assuming X is your feature matrix
# Initialize the scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()

# Apply standardization
X_standardized = scaler_standard.fit_transform(X)

# Apply normalization
X_normalized = scaler_minmax.fit_transform(X)

In [ ]:
# Assuming 'lulc_data' and 'gdp_data' are your LULC and GDP datasets
# Assume both datasets have 'Year' and 'Country' columns

# Check common time periods
common_years = set(lulc_data['Year']).intersection(gdp_data['Year'])

# Filter datasets to include only common time periods
lulc_data_filtered = lulc_data[lulc_data['Year'].isin(common_years)]
gdp_data_filtered = gdp_data[gdp_data['Year'].isin(common_years)]


# NTL

In [ ]:

# Replace these paths with your actual file paths
file_paths = [
    "C:\\Users\\my\\Documents\\topic17\\topic17\\2018_MosaickData.tif",
    "C:\\Users\\my\\Documents\\topic17\\topic17\\2019_MosaickData.tif",
    "C:\\Users\\my\\Documents\\topic17\\topic17\\2020_MosaickData.tif",
    "C:\\Users\\my\\Documents\\topic17\\topic17\\2021_MosaickData.tif",
    "C:\\Users\\my\\Documents\\topic17\\topic17\\2018_MosaickData.tif"
]


In [ ]:
# Read the first four files for training
training_file_paths = file_paths[:-1]
testing_file_path = file_paths[-1]


In [ ]:
# Read and flatten each training NTL file
ntl_data_list = []
for i, file_path in enumerate(training_file_paths):
    with rasterio.open(file_path) as src:
        ntl_data_list.append(src.read().flatten())

In [ ]:
# Replace this list with your actual GDP growth data
gdp_growth_data = [135.12, 145.60, 134.52, 197.00]

In [ ]:
'''# Ensure both arrays have the same length
#min_length = min(len(gdp_growth_data), len(ntl_data_list[0]))
ntl_data_array[0] = ntl_data_list[0][:min_length]
gdp_growth_array = gdp_growth_data[:min_length]'''

# Ensure both arrays have the same length
min_length = min(len(gdp_growth_data), len(ntl_data_list[0])) if ntl_data_list else 0

# Check if ntl_data_list has elements before accessing index 0
if ntl_data_list and len(ntl_data_list[0]) > 0:
    ntl_data_list[0] = ntl_data_list[0][:min_length]
    gdp_growth_data = gdp_growth_data[:min_length]
else:
    print("Error: ntl_data_list is empty or does not have elements at index 0.")


In [ ]:
# Step 3: Create a DataFrame
data = {'GDP_Growth': gdp_growth_data}

# Check if ntl_data_list is not empty and has at least one array
if ntl_data_list and len(ntl_data_list[0]) > 0:
    for i in range(len(ntl_data_list[0])):
        data[f'NTL_Value_{i+1}'] = [arr[i] for arr in ntl_data_list]

# Create DataFrame
df = pd.DataFrame(data)




In [ ]:
# Step 4: Handling Missing Values (if any)
df.dropna(inplace=True)


In [ ]:
# Step 5: Prepare Data for Training
X = df.drop(columns=['GDP_Growth'])
y = df['GDP_Growth']


In [ ]:
# Step 6: Split Data into Training and Testing Sets (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Step 7: Train a Random Forest Regressor Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



In [ ]:
# Step 8: Make Predictions on the Test Set
y_test_pred = model.predict(X_test)



In [ ]:
# Step 9: Evaluate the Model on the Test Set
mse_test = mean_squared_error(y_test, y_test_pred)
print(f'Mean Squared Error on Test Set: {mse_test}')




In [ ]:
# Step 10: Make Predictions for New NTL Data
with rasterio.open(testing_file_path) as src:
    new_ntl_data = src.read().flatten()

new_ntl_data_flattened = new_ntl_data.flatten()
scaler = StandardScaler()
scaler.fit(X_train)
new_ntl_data_scaled = scaler.transform(new_ntl_data_flattened.reshape(1, -1))
new_predictions_gdp = model.predict(new_ntl_data_scaled)

# Print the predictions for the new NTL data
print("Predictions for New NTL Data (GDP):")


In [ ]:
# Replace these paths with the actual file paths of your GeoTIFF files
lulc_2012_path = "C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2012.tif"
lulc_2013_path = "C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2013.tif"
lulc_2014_path = "C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2014.tif"
lulc_2015_path = "C:\\Users\\my\\Documents\\UttarPradesh\\LULC\\UP_LULC_2015.tif"

# Read the GeoTIFF files using rasterio
with rasterio.open(lulc_2012_path) as src_2012, \
        rasterio.open(lulc_2013_path) as src_2013, \
        rasterio.open(lulc_2014_path) as src_2014, \
        rasterio.open(lulc_2015_path) as src_2015:
    
    # Read raster data as numpy arrays and flatten
    lulc_2012 = src_2012.read(1).flatten()
    lulc_2013 = src_2013.read(1).flatten()
    lulc_2014 = src_2014.read(1).flatten()
    lulc_2015 = src_2015.read(1).flatten()

# Plotting LULC and GDP
plt.figure(figsize=(12, 6))

# Scatter plot for LULC vs GDP for each year
plt.scatter(lulc_2012, lulc_2013, label='2012-13')
plt.scatter(lulc_2013, lulc_2014, label='2013-14')
plt.scatter(lulc_2014, lulc_2015, label='2014-15')

plt.title('LULC vs GDP for Uttar Pradesh')
plt.xlabel('LULC')
plt.ylabel('GDP')
plt.legend()
plt.show()